In [20]:
'''
>>> Script for converting numbers to labels 


'''


import pandas as pd

input_path = "/Users/arashalborz/Desktop/amiv_nlp_2025/processed_data/combined_author_embeddings_with_liwc.csv"
df = pd.read_csv(input_path)

def score_to_label(score):
    if score <= 32:
        return "Low"
    elif score <= 66:
        return "Medium"
    else:
        return "High"

traits = ["openness", "conscientiousness", "extraversion", "agreeableness", "neuroticism"]

for trait in traits:
    df[trait] = df[trait].apply(lambda x: score_to_label(x * 100))  # scale back to 0–100

output_path = "/Users/arashalborz/Desktop/amiv_nlp_2025/processed_data/combined_author_embeddings_with_liwc_labeled.csv"
df.to_csv(output_path, index=False)
print(f"✅ Saved labeled trait data to {output_path}")

✅ Saved labeled trait data to /Users/arashalborz/Desktop/amiv_nlp_2025/processed_data/combined_author_embeddings_with_liwc_labeled.csv


In [19]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, f1_score, classification_report

# Load the data
df = pd.read_csv("/Users/arashalborz/Desktop/amiv_nlp_2025/processed_data/combined_author_embeddings_with_liwc_labeled.csv")

# --- Prepare input (X) and label (y: only openness) ---
X = df[[col for col in df.columns if col.startswith("embed_") or col.startswith("liwc_")]].values
y = df["openness"].values  # Categorical label: Low, Medium, High

# --- Split ---
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# --- Normalize features ---
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# --- Train SVM ---
svm = SVC(kernel="linear", C=1.0, random_state=42)
svm.fit(X_train_scaled, y_train)

# --- Predict ---
y_pred = svm.predict(X_test_scaled)

# --- Evaluation ---
accuracy = accuracy_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred, average="macro")
report = classification_report(y_test, y_pred, zero_division=0)

print("SVM Openness - Accuracy:", accuracy)
print("SVM Openness - F1-score (macro):", f1)
print("Classification Report:\n", report)

# Save report to file
with open("svm_classification_report_openness.txt", "w") as f:
    f.write("SVM Classification Report for Openness:\n")
    f.write(report)

print("✅ Saved to svm_classification_report_openness.txt")

SVM Openness - Accuracy: 0.4426751592356688
SVM Openness - F1-score (macro): 0.3575830567902518
Classification Report:
               precision    recall  f1-score   support

        High       0.55      0.60      0.58       174
         Low       0.22      0.21      0.22        48
      Medium       0.31      0.26      0.28        92

    accuracy                           0.44       314
   macro avg       0.36      0.36      0.36       314
weighted avg       0.43      0.44      0.43       314

✅ Saved to svm_classification_report_openness.txt


In [6]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, f1_score, classification_report

# Load the labeled data
df = pd.read_csv("combined_author_embeddings_with_liwc_labeled.csv")

# Prepare feature matrix X
X = df[[col for col in df.columns if col.startswith("embed_") or col.startswith("liwc_")]].values

# Trait columns
traits = ["openness", "conscientiousness", "extraversion", "agreeableness", "neuroticism"]

# Store results
reports = []

# Loop over each trait
for trait in traits:
    print(f"\n==== Trait: {trait.capitalize()} ====")

    # Labels
    y = df[trait].values

    # Train/test split
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Normalize features
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    # Train SVM
    svm = SVC(kernel="linear", C=1.0, random_state=42)
    svm.fit(X_train_scaled, y_train)

    # Predict
    y_pred = svm.predict(X_test_scaled)

    # Evaluation
    accuracy = accuracy_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred, average="macro")
    report = classification_report(y_test, y_pred, zero_division=0)

    print("Accuracy:", accuracy)
    print("F1-score (macro):", f1)
    print("Classification Report:\n", report)

    # Save each report
    reports.append((trait, accuracy, f1, report))

# Save all reports to file
with open("svm_classification_report_all_traits.txt", "w") as f:
    for trait, acc, f1, rep in reports:
        f.write(f"=== Trait: {trait.capitalize()} ===\n")
        f.write(f"Accuracy: {acc:.4f}\n")
        f.write(f"F1-score (macro): {f1:.4f}\n")
        f.write(rep + "\n\n")

print("\n✅ All SVM classification reports saved to svm_classification_report_all_traits.txt")


==== Trait: Openness ====
Accuracy: 0.4426751592356688
F1-score (macro): 0.3575830567902518
Classification Report:
               precision    recall  f1-score   support

        High       0.55      0.60      0.58       174
         Low       0.22      0.21      0.22        48
      Medium       0.31      0.26      0.28        92

    accuracy                           0.44       314
   macro avg       0.36      0.36      0.36       314
weighted avg       0.43      0.44      0.43       314


==== Trait: Conscientiousness ====
Accuracy: 0.4267515923566879
F1-score (macro): 0.4098462519212225
Classification Report:
               precision    recall  f1-score   support

        High       0.35      0.47      0.40        85
         Low       0.52      0.47      0.50       146
      Medium       0.37      0.30      0.33        83

    accuracy                           0.43       314
   macro avg       0.41      0.41      0.41       314
weighted avg       0.44      0.43      0.43       

In [7]:
'''

RANDOM FOREST

'''


import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, f1_score, classification_report

# Load the labeled data
df = pd.read_csv("combined_author_embeddings_with_liwc_labeled.csv")

# Prepare feature matrix X (all LIWC + BERT embeddings)
X = df[[col for col in df.columns if col.startswith("embed_") or col.startswith("liwc_")]].values

# Target labels (Openness)
y = df["openness"].values  # Should be "Low", "Medium", or "High"

# Train/test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Normalize features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Train Random Forest
rf = RandomForestClassifier(n_estimators=100, random_state=42, n_jobs=-1)
rf.fit(X_train_scaled, y_train)

# Predict
y_pred = rf.predict(X_test_scaled)

# Evaluate
accuracy = accuracy_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred, average="macro")
report = classification_report(y_test, y_pred, zero_division=0)

# Print results
print("Openness - Accuracy:", accuracy)
print("Openness - F1-score (macro):", f1)
print("Classification Report:\n", report)

# Save report
with open("random_forest_openness_report.txt", "w") as f:
    f.write("Classification Report - Random Forest (Openness):\n")
    f.write(report)

print("✅ Saved report to random_forest_openness_report.txt")

Openness - Accuracy: 0.5127388535031847
Openness - F1-score (macro): 0.2937782654459396
Classification Report:
               precision    recall  f1-score   support

        High       0.56      0.86      0.67       174
         Low       0.09      0.02      0.03        48
      Medium       0.31      0.12      0.17        92

    accuracy                           0.51       314
   macro avg       0.32      0.33      0.29       314
weighted avg       0.41      0.51      0.43       314

✅ Saved report to random_forest_openness_report.txt


In [8]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, f1_score, classification_report

# Load the labeled dataset
df = pd.read_csv("combined_author_embeddings_with_liwc_labeled.csv")

# Input features
X = df[[col for col in df.columns if col.startswith("embed_") or col.startswith("liwc_")]].values

# Define traits to classify
traits = ["openness", "conscientiousness", "extraversion", "agreeableness", "neuroticism"]

# Normalize features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Store evaluation results
all_reports = ""

for trait in traits:
    print(f"\n========== Predicting {trait.capitalize()} ==========")
    
    # Labels
    y = df[trait].values
    
    # Train/test split
    X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)
    
    # Train model
    clf = RandomForestClassifier(n_estimators=100, random_state=42, n_jobs=-1)
    clf.fit(X_train, y_train)
    
    # Predict
    y_pred = clf.predict(X_test)
    
    # Metrics
    acc = accuracy_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred, average="macro")
    report = classification_report(y_test, y_pred, zero_division=0)
    
    # Output
    print(f"Accuracy: {acc:.4f}")
    print(f"F1-score (macro): {f1:.4f}")
    print(report)
    
    all_reports += f"\n\n=== {trait.upper()} ===\n"
    all_reports += f"Accuracy: {acc:.4f}\nF1-score (macro): {f1:.4f}\n"
    all_reports += report

# Save all reports to file
with open("random_forest_all_traits_report.txt", "w") as f:
    f.write("Classification Reports - Random Forest (All Traits)\n")
    f.write(all_reports)

print("\n✅ Saved full trait classification report to random_forest_all_traits_report.txt")


========== Predicting Openness ==========
Accuracy: 0.5127
F1-score (macro): 0.2938
              precision    recall  f1-score   support

        High       0.56      0.86      0.67       174
         Low       0.09      0.02      0.03        48
      Medium       0.31      0.12      0.17        92

    accuracy                           0.51       314
   macro avg       0.32      0.33      0.29       314
weighted avg       0.41      0.51      0.43       314


========== Predicting Conscientiousness ==========
Accuracy: 0.4427
F1-score (macro): 0.2882
              precision    recall  f1-score   support

        High       0.40      0.07      0.12        85
         Low       0.47      0.86      0.61       146
      Medium       0.24      0.10      0.14        83

    accuracy                           0.44       314
   macro avg       0.37      0.34      0.29       314
weighted avg       0.39      0.44      0.35       314


========== Predicting Extraversion ==========
Accuracy: 0.